                                 利用MLP进行保险客户的画像分析以及购买理财产品的意愿预测

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

一、导入数据

In [5]:
dataset=pd.read_csv(r"E:\学习资料\金融大数据\保险业精准营销模型\Insurance.csv")
dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X77,X78,X79,X80,X81,X82,X83,X84,X85,Y
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,0


二、数据预处理

In [7]:
dataset.describe

<bound method NDFrame.describe of       X1  X2  X3  X4  X5  X6  X7  X8  X9  X10  ...  X77  X78  X79  X80  X81  \
0     33   1   3   2   8   0   5   1   3    7  ...    0    0    0    1    0   
1     37   1   2   2   8   1   4   1   4    6  ...    0    0    0    1    0   
2     37   1   2   2   8   0   4   2   4    3  ...    0    0    0    1    0   
3      9   1   3   3   3   2   3   2   4    5  ...    0    0    0    1    0   
4     40   1   4   2  10   1   4   1   4    7  ...    0    0    0    1    0   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...   
5817  36   1   1   2   8   0   6   1   2    1  ...    0    0    0    1    0   
5818  35   1   4   4   8   1   4   1   4    6  ...    0    0    0    1    0   
5819  33   1   3   4   8   0   6   0   3    5  ...    0    0    0    1    0   
5820  34   1   3   2   8   0   7   0   2    7  ...    0    0    0    0    0   
5821  33   1   3   3   8   0   6   1   2    7  ...    0    0    0    0    0   

      X82  X83  X

In [13]:
dataset.isnull().sum()

X1     0
X2     0
X3     0
X4     0
X5     0
      ..
X82    0
X83    0
X84    0
X85    0
Y      0
Length: 86, dtype: int64

In [15]:
dataset.shape

(5822, 86)

In [19]:
dataset=dataset.dropna()
dataset.shape

(5822, 86)

In [25]:
##查看数据是否均衡
dataset.groupby("Y").count()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85
Y,,,,,,,,,,,,,,,,,,,,,
0,5474,5474,5474,5474,5474,5474,5474,5474,5474,5474,...,5474,5474,5474,5474,5474,5474,5474,5474,5474,5474
1,348,348,348,348,348,348,348,348,348,348,...,348,348,348,348,348,348,348,348,348,348


In [30]:
cols=dataset.keys()
cols

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31',
       'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41',
       'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51',
       'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61',
       'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71',
       'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78', 'X79', 'X80', 'X81',
       'X82', 'X83', 'X84', 'X85', 'Y'],
      dtype='object')

In [32]:
cols_x=cols[:-1]
cols_y=cols[-1]
X=dataset[cols_x]
Y=dataset[cols_y]
X.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,0,1,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,0,1,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,0,1,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,0,1,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,0,1,0,0,0,0,0


In [40]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.8)

In [59]:
Xtrain=torch.tensor(x_train.to_numpy(),dtype=torch.float32)
Ytrain=torch.tensor(y_train.to_numpy(),dtype=torch.float32)
Xtest=torch.tensor(x_test.to_numpy(),dtype=torch.float32)

三、模型构建

In [47]:
##定义MLP多层感知机模型
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MLP, self).__init__()
        self.linear1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.linear2=nn.Linear(hidden_size,output_size)
    
    def forward(self,x):
        out=self.linear1(x)
        out=self.relu(out)
        out=self.linear2(out)
        return out

In [63]:
Ytrain=Ytrain.view(Ytrain.shape[0],1)##因为下面的经过forward()计算得到的output是一个二维矩阵所以要把一维向量Ytrain也变成二维的！！！
input_size=Xtrain.shape[1]
hidden_size=50
output_size=1
input_size

85

In [53]:
model=MLP(input_size,hidden_size,output_size)
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e3)

In [79]:
##模型训练
epochs=1000
for epoch in range(epochs):
    model.train()
    output=model.forward(Xtrain)
    loss=criterion(output,Ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1)%100==0:
        print(f"第{epoch+1}次训练。训练误差为：{loss}")

第100次训练。训练误差为：44.22959518432617
第200次训练。训练误差为：36.005157470703125
第300次训练。训练误差为：36.14932632446289
第400次训练。训练误差为：34.17472839355469
第500次训练。训练误差为：25.575428009033203
第600次训练。训练误差为：23.743215560913086
第700次训练。训练误差为：20.20291519165039
第800次训练。训练误差为：11.297861099243164
第900次训练。训练误差为：7.8614702224731445
第1000次训练。训练误差为：3.0499396324157715


In [117]:
##模型预测与评估
model.eval()
with torch.no_grad():
    outputs = model(Xtest)
    _, predicted = torch.max(outputs, 1)

In [119]:
predicted=predicted.tolist()

In [130]:
##这里y_test,predicted变量都是一维向量
score=accuracy_score(y_test,predicted)
print(f"预测准确度为：{score}")

预测准确度为：0.9424892703862661
